In [2]:
import os
import copy
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.io
import matplotlib.pyplot as plt
import glob
import re
from datetime import datetime
from dateutil import parser
import dateutil.parser
from scipy.io import savemat

import numpy as np
import scipy.misc
import numpy.random as rng
from PIL import Image, ImageDraw, ImageFont
from sklearn.utils import shuffle
import nibabel as nib #reading MR images
import math
from matplotlib import pyplot as plt
import statistics

In [2]:
FREESURFER_A_USAR = 'freesurfer_cross_long_a_usar.csv'
ASEG_STATS = 'aseg_stats.csv'
LIPIDOMICS = 'lipidomics_v3.csv'
NAO_PROCESSADOS = 'nao_processados_a_usar_melhores_expandido.csv'
DATASET = 'DatasetPro4.0.csv'
MMSE = 'MMSE.csv'

In [5]:
volumes= pd.read_csv('rh_aparc.DKTatlas_stats.csv')
# lipidomics = pd.read_csv('D:\Tese\lipidomics_v3.csv')

rid = []
viscode = []
for index, row in volumes.iterrows():
    match = re.search(r"S_0?(.*?)\.", row['rh.aparc.DKTatlas.volume']).group(1)
    match2 = re.search(r"\.(.*?)$", row['rh.aparc.DKTatlas.volume']).group(1)
    rid.append(match)
    viscode.append(match2)
volumes.pop('rh.aparc.DKTatlas.volume')
volumes.insert(0, 'RID', rid)
volumes.insert(1, 'VISCODE', viscode)
volumes.to_csv('rh_aparc.DKTatlas_stats1.0.csv', index=False)

In [ ]:
subject_list= pd.read_csv('D:\Tese\SubjectList.csv')
dataset11= pd.read_csv('D:\Tese\Dataset_Final\Dataset1.1.csv')

age = []
sex = []
date = []
group = []
for index1, row1 in dataset11.iterrows():
    for index2, row2 in subject_list.iterrows():
        if (row1['RID'] == row2['RID']) and (row1['VISCODE'] == row2['VISCODE']):
            age.append(row2['Age'])
            sex.append(row2['Sex'])
            date.append(row2['Acq Date'])
            group.append(row2['Group'])
dataset11.insert(2, 'Age', age)
dataset11.insert(3, 'Sex', sex)
dataset11.insert(4, 'MRI_Date', date)
dataset11.insert(5, 'Group', group)
dataset11.to_csv('Dataset1.2.csv', index=False)

In [ ]:
dataset12= pd.read_csv('D:\Tese\Dataset_Final\Dataset1.2.csv')
lipidomics = pd.read_csv('D:\Tese\lipidomics_v3.csv')

dataset13 = pd.merge(dataset12, lipidomics, on=['RID', 'VISCODE'], how='inner')
dataset13 = dataset13.drop_duplicates(subset=['RID', 'VISCODE'])
lipidomics_date = dataset13.pop('EXAMDATE')
viscode2 = dataset13.pop('VISCODE2')
dataset13.insert(6, 'EXAMDATE', lipidomics_date)
dataset13.insert(7, 'VISCODE2', viscode2)
dataset13 = dataset13.sort_values(by=['RID', 'MRI_Date'], ascending=[True, True])
dataset13.to_csv('Dataset1.3.csv', index=False)

In [6]:
merged = pd.read_csv('merged.csv')
merged = merged.sort_values(by=['RID', 'Exam_Date'], ascending=[True, True])
merged.to_csv('merged_final.csv', index=False)

In [11]:
dataset= pd.read_csv('DatasetPro6.0.csv')
rh= pd.read_csv('rh_aparc.DKTatlas_stats.csv')
lh= pd.read_csv('lh_aparc.DKTatlas_stats.csv')
merged_aparc = pd.merge(rh, lh, on=['RID', 'VISCODE'], how='inner')
merged_aparc = merged_aparc.drop(columns=['BrainSegVolNotVent_x', 'eTIV_x', 'BrainSegVolNotVent_y', 'eTIV_y'])
merged_df = pd.merge(dataset, merged_aparc, on=['RID', 'VISCODE'], how='inner')
merged_df.to_csv('DatasetPro7.0.csv', index=False)

In [16]:
dataset= pd.read_csv('DatasetPro7.0.csv')
count = 0
for col in dataset.columns[1639:1704]:
    term = dataset.pop(col)
    dataset.insert(9+count, col, term)
    count +=1
dataset.to_csv('DatasetPro8.0.csv', index=False)

### Total Intracranial Volume Ratio

In [ ]:
dataset = pd.read_csv(DATASET)
dataset['LHC_ICV'] = None
dataset['RHC_ICV'] = None
for index, row in dataset.iterrows():
        l = round(row['Left-Hippocampus']/row['EstimatedTotalIntraCranialVol'],9)
        r = round(row['Right-Hippocampus']/row['EstimatedTotalIntraCranialVol'],9)
        dataset.at[index, 'LHC_ICV'] = l
        dataset.at[index, 'RHC_ICV'] = r
        print(dataset.loc[index, 'RID'])

dataset.to_csv('DatasetPro2.0.csv', index=False)

### Add MMSE Values to Dataset

In [ ]:
dataset = pd.read_csv('DatasetPro2.0.csv')
mmse = pd.read_csv(MMSE)
dataset['MMSE'] = None
for index, row in dataset.iterrows():
    for index2, row2 in mmse.iterrows():
        if (row['RID'] == row2['RID']) and (row['VISCODE2'] == row2['VISCODE2']):
            dataset.at[index, 'MMSE'] = row2['MMSCORE']
            print(dataset.loc[index, 'RID'])
dataset.to_csv('DatasetPro3.0.csv', index=False)


### Normalizacao de Volumes

In [ ]:
dataset = pd.read_csv('DatasetPro8.0.csv')

for index, row in dataset.iterrows():
    for i in dataset.columns[9:132]:
        value = round(row[i]/row['EstimatedTotalIntraCranialVol'],9)
        new_col = i+'_ICV'
        dataset.at[index, new_col] = value
dataset.to_csv('DatasetPro9.0.csv', index=False)  


# print(dataset.columns[9:132])

### Normalizacao de Lipidos

In [8]:
DATASET = 'DatasetPro3.0.csv'

In [ ]:
dataset = pd.read_csv(DATASET)

for index, row in dataset.iterrows():
    total = row[dataset.columns[74:855]].sum()
    for i in dataset.columns[74:855]:
        new_col = i+'%mol'
        perc_mol = (row[i]/total) * 100
        dataset.at[index, new_col] = perc_mol
dataset.to_csv('DatasetPro4.0.csv', index=False)    

### MMSE Missing Values Handling

In [27]:
DATASET = 'DatasetPro5.0.csv'
dataset = pd.read_csv(DATASET)

group_items = []
for index, row in dataset.iterrows():
    if row['Group'] not in group_items:
        group_items.append(row['Group'])
print(group_items)

#---------------------------------------------------------------------------------------------------------------
valuesAD = dataset.loc[dataset['Group'] == 'AD', 'MMSE'].tolist()
ad = [x for x in valuesAD if not np.isnan(x)]
median_value_ad = statistics.median(ad)

valuesMCI = dataset.loc[dataset['Group'] == 'MCI', 'MMSE'].tolist()
mci = [x for x in valuesMCI if not np.isnan(x)]
median_value_mci = statistics.median(mci)

valuesCN = dataset.loc[dataset['Group'] == 'CN', 'MMSE'].tolist()
cn = [x for x in valuesCN if not np.isnan(x)]
median_value_cn = statistics.median(cn)

valuesEMCI = dataset.loc[dataset['Group'] == 'EMCI', 'MMSE'].tolist()
emci = [x for x in valuesEMCI if not np.isnan(x)]
median_value_emci = statistics.median(emci)

valuesLMCI = dataset.loc[dataset['Group'] == 'LMCI', 'MMSE'].tolist()
lmci = [x for x in valuesLMCI if not np.isnan(x)]
median_value_lmci = statistics.median(lmci)

valuesSMC = dataset.loc[dataset['Group'] == 'SMC', 'MMSE'].tolist()
smc = [x for x in valuesSMC if not np.isnan(x)]
median_value_smc = statistics.median(smc)

print(median_value_ad,median_value_mci,median_value_cn,median_value_emci,median_value_lmci,median_value_smc)

#-----------------------------------------------------------------------------------------------------------

dataset.loc[dataset['Group'] == 'AD', 'MMSE'] = dataset.loc[dataset['Group'] == 'AD', 'MMSE'].fillna(median_value_ad)
dataset.loc[dataset['Group'] == 'MCI', 'MMSE'] = dataset.loc[dataset['Group'] == 'MCI', 'MMSE'].fillna(median_value_mci)
dataset.loc[dataset['Group'] == 'CN', 'MMSE'] = dataset.loc[dataset['Group'] == 'CN', 'MMSE'].fillna(median_value_cn)
dataset.loc[dataset['Group'] == 'EMCI', 'MMSE'] = dataset.loc[dataset['Group'] == 'EMCI', 'MMSE'].fillna(median_value_emci)
dataset.loc[dataset['Group'] == 'lmci', 'MMSE'] = dataset.loc[dataset['Group'] == 'lmci', 'MMSE'].fillna(median_value_lmci)
dataset.loc[dataset['Group'] == 'SMC', 'MMSE'] = dataset.loc[dataset['Group'] == 'SMC', 'MMSE'].fillna(median_value_smc)
dataset.to_csv('DatasetPro6.0.csv', index=False)  

['AD', 'MCI', 'CN', 'EMCI', 'LMCI', 'SMC']
22.0 26.0 29.0 29.0 26.0 29.0
